# AI plugins
After Lab 1, you should have grasped how AI models expand their capabilities to interact with the external world through plugins. Plugins serve as the foundational building blocks of the Semantic Kernel. They allow you to encapsulate capabilities into a single unit of functionality that the kernel can then execute 
[Read more](https://learn.microsoft.com/en-us/semantic-kernel/agents/plugins/?tabs=Csharp)

At a high-level, a plugin is a group of functions which semantical description. 

You can create tow types of functions:
    
    - prompts 
    - native functions.

## Native functions
With native functions, we empower AI services to perform tasks equivalent to those of any other applications, be it web APIs or console applications, but in an intelligent manner.

The [LightPlug](plugins/LightPlugin.cs) we used in previous lab is an example of native functions.
It has annotations to decribe the functions, which can be used by AI service to understand how the functions behave.

Here, we are going to define another plugin that enable AI service to seach files in local directories and read PDF files.

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.3.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.3.1-alpha"
#!import config/Settings.cs 
#!import lib/Usings.cs
#!import lib/Utilities.cs

Installed Packages Microsoft.SemanticKernel, 1.3.1 Microsoft.SemanticKernel.Plugins.Core, 1.3.1-alpha

### Method Attributes 

[KernelFunction] and [Description] are required attributes for the methods you want an AI to call your code. They are serialized into metadata so that an AI service can understand what tools are available to use.

In [2]:
#r "nuget: pdfpig, 0.1.8"
using UglyToad.PdfPig;
using UglyToad.PdfPig.Content;
using System.Collections;
using System.Collections.Generic;

public sealed class PdfFilesPlugin
{
    [KernelFunction, Description("Get the list of files in a directory")] 
    public static string[] GetFiles([Description("The directory to search for files")]string directory)
    {
        return Directory.GetFiles(directory);
    }

    [KernelFunction, Description("Get the list of files in a directory with a specific pattern")]
    public static string[] GetFilesWithPattern([Description("The directory to search for files")]string directory, [Description("The pattern to search for")]string pattern)
    {
        return Directory.GetFiles(directory, pattern);
    }

    [KernelFunction, Description("Read the content of a pdf file")]
    public static string ReadPdf([Description("The path to the pdf file")]string path)
    {
        using (var document = PdfDocument.Open(path))
        {
            var sb = new StringBuilder();
            foreach (var page in document.GetPages())
            {
                sb.AppendLine(page.Text);
            }
            return sb.ToString();
        }
    }
}

Installed Packages pdfpig, 0.1.8

## Prompt functions
The "Prompt" function, as the name suggests, encapsulates templated prompts for interacting with AI services. These prompts contain placeholders, such as {{input}}, which represent parameters for the task. The kernel utilizes a template engine to accept these parameters and generate the final prompt text, which is then sent to the AI service.

This "Prompt" function consists of two files: skprompt.txt, which serves as the prompt template, and config.json, which describes this plugin. The configuration file specifies the function's input parameters, provides a description to semantically define the function, and defines the configuration for the AI service.

```
[SUMMARIZATION RULES]
DONT WASTE WORDS
USE SHORT, CLEAR, COMPLETE SENTENCES.
DO NOT USE BULLET POINTS OR DASHES.
USE ACTIVE VOICE.
MAXIMIZE DETAIL, MEANING
FOCUS ON THE CONTENT

[BANNED PHRASES]
This article
This document
This page
This material
[END LIST]

Summarize:
Hello how are you?
+++++
Hello

Summarize this
{{$input}}
+++++
```

``` json
{
  "schema": 1,
  "description": "Summarize given text or any text document",
  "execution_settings": {
    "default": {
      "max_tokens": 512,
      "temperature": 0.0,
      "top_p": 0.0,
      "presence_penalty": 0.0,
      "frequency_penalty": 0.0
    }
  },
  "input_variables": [
    {
      "name": "input",
      "description": "Text to summarize",
      "default": "",
      "is_required": true
    }
  ]
}
```

### Now lets put the plugins we just created into work.

In [3]:
#r "nuget: Microsoft.SemanticKernel, 1.3.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.3.1-alpha"
#!import config/Settings.cs 
#!import lib/Usings.cs
#!import lib/Utilities.cs


Installed Packages Microsoft.SemanticKernel, 1.3.1 Microsoft.SemanticKernel.Plugins.Core, 1.3.1-alpha

In [4]:
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
var builder = Kernel.CreateBuilder();

//register ai serivces
builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);

//register plugins
#pragma warning disable SKEXP0050
builder.Plugins.AddFromPromptDirectory("./plugins/");
builder.Plugins.AddFromType<PdfFilesPlugin>();

var kernel = builder.Build();


In [6]:
var input = "summarize the pdf file at ./pdf";

ChatHistory history = [];

// Get chat completion service
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

    // Get user input
Console.WriteLine($"User > {input}");
history.AddUserMessage(input);

// Enable auto function calling
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new()
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

// Get the response from the AI
var result = await chatCompletionService.GetChatMessageContentAsync(
        history,
        executionSettings: openAIPromptExecutionSettings,
        kernel: kernel);

// Add the message from the agent to the chat history
history.AddMessage(result.Role, result.Content);

// Print the results
Console.WriteLine("Assistant > " + result.Content);


User > summarize the pdf file at ./pdf


"The Happy Prince" by Oscar Wilde tells the story of a magnificent statue, admired by the citizens for its beauty and splendor, standing high above the city. One night, a lonely Swallow, who had stayed behind after his companions flew to Egypt, seeks shelter between the statue's feet. As he prepares to sleep, he is surprised by a drop of water, despite the clear sky, marking the beginning of an unexpected interaction.

Assistant > Microsoft.DotNet.Interactive.DisplayedValue


## Your Turn

Now it's your turn to try out the plugins. Can you add a new plug that translates the summarization into French(hint: prompt function), and anther to save summarization output into a text file(hint: native function)? 